In [18]:
import numpy as np
import pandas as pd
import polars as pl

# See readme for how I obtained the the below data
#catalog = pd.read_csv('../data/1680058493237O-result.csv', sep=',', header=0, dtype=np.float64)

# smaller sampe of the above data for testing purposes
catalog = pd.read_csv('../data/test.csv', sep=',', header=0, dtype=np.float64)

# arbitrary threashold 'k is for Kalle'
threshold_k = np.pi / 4

# helper functio
deg2rad = lambda rad: rad * np.pi / 180

# sort the catalog by magnitudes
stars = catalog.sort_values(by='b1mag')

stars

,ra,dec,b1mag
0,101.287024,-16.716001,-1.50
1,95.987955,-52.695661,-0.62
2,219.899787,-60.835273,0.00
3,213.915319,19.182450,0.00
4,279.234737,38.783690,0.03
...,...,...,...
53,342.890496,71.612316,11.99
54,56.548597,71.616219,11.99
55,26.345215,66.635347,11.99
45,84.386296,75.033424,11.99


In [19]:
# This creates a new vector of tuples with precomputed 'rays' for each star
#   0 = cos(dec) * cos(ra)
#   1 = cos(dec) * sin(ra)
#   2 = sin(dec)  
rays = np.stack(
            (   np.cos(deg2rad(stars['dec'])) * np.cos(deg2rad(stars['ra'])), 
                np.cos(deg2rad(stars['dec'])) * np.sin(deg2rad(stars['ra'])), 
                np.sin(deg2rad(stars['dec']))
            ) 
        ).T
rays

array([[-0.18745319,  0.93921852, -0.287628  ],
       [-0.06322262,  0.60274195, -0.79542759],
       [-0.37385775, -0.31259124, -0.87322225],
       [-0.78378669, -0.52698704,  0.32857736],
       [ 0.12509649, -0.76941311,  0.62638194],
       [ 0.19505172,  0.97036265, -0.14265749],
       [ 0.13050036,  0.68231601,  0.71931531],
       [ 0.02088846,  0.99143518,  0.12891845],
       [-0.41811213,  0.9038191 ,  0.09106749],
       [ 0.49272291,  0.22380426, -0.84091366],
       [-0.42393818, -0.2542838 , -0.86926185],
       [ 0.55971537, -0.82768132, -0.04077179],
       [ 0.34390209,  0.89497374,  0.28417136],
       [ 0.45922089, -0.87484256,  0.15416441],
       [-0.34481839, -0.82641108, -0.44513482],
       [-0.91408008, -0.3563536 , -0.19357096],
       [-0.39151333,  0.79116116,  0.46987374],
       [ 0.83733252, -0.23358762, -0.49427833],
       [ 0.45564906, -0.53618207,  0.71055804],
       [-0.44940405, -0.0523942 , -0.89179081],
       [-0.49379562, -0.10433183, -0.863

Now we have and array of tuples with precalulated data for each star in the catalog.  Magnitude is ignored. 

In [24]:
numrows = rays.shape[0]

rays[0,:]  # get the row at position 0 of the rays collection
rays[0+1,:] # gets the next row of the rays collection  

#rays[0+1:,:] # this is where I loose track?  
d = np.arccos((rays[0,:] * rays[0+1:,:]).sum(-1) )

flat = np.flatnonzero(d < threshold_k)
flat


array([0.63217378, 1.54314028, 2.02922629, 2.75516619, 0.41317207,
       1.14890584, 0.47306036, 0.44857381, 1.20284039, 1.48000709,
       2.62714969, 0.80324569, 2.83080424, 2.19384049, 1.6786757 ,
       0.821239  , 1.8071736 , 2.48714025, 1.27496348, 1.32546123,
       1.00930566, 0.22062815, 1.29403559, 1.80005261, 2.1572562 ,
       0.85817382, 0.53035538, 1.00640781, 0.40124361, 1.77538001,
       1.71532662, 1.39462182, 0.6145055 , 1.63296906, 1.85128464,
       1.72003298, 2.11650022, 1.7691714 , 2.26242708, 2.09692828,
       1.82847205, 2.25118892, 2.16152083, 1.43134974, 2.27045301,
       1.76612094, 2.15791915, 1.77144455, 1.5683638 , 1.77075465,
       1.47049987, 2.08782701, 2.29237282, 1.92535686, 1.96957926,
       2.30009161, 2.02494451, 1.69664096, 1.54038168, 2.26802917,
       1.82047003, 2.16415893, 2.01615039, 1.71310238, 1.81393419,
       1.62733941, 1.75430071, 1.52273926, 1.99306838, 1.82574623,
       1.49851373, 1.58473718, 1.85709869, 1.69000716, 1.90413

In [26]:
i_, j_, d_ = [],[],[]
for i in range(0, numrows):
    d = np.arccos((rays[i,:] * rays[i+1:,:]).sum(-1) )
    for j in np.flatnonzero(d < threshold_k):
        i_.append(i)
        j_.append(i+1+j)
        d_.append(d)

pairs = pd.DataFrame({'i':np.array(i_), 'j':np.array(j_), 'd':np.array(d_)}) #.sort_values(by='d')

pairs


/tmp/ipykernel_597247/2262240069.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  pairs = pd.DataFrame({'i':np.array(i_), 'j':np.array(j_), 'd':np.array(d_)}).sort_values(by='d')


ValueError: operands could not be broadcast together with shapes (38,) (82,) 